In [2]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [66]:
NUM_RANDOM_SEEDS = 5
MODEL_CLASS = "lr"
EPSILON = 0.05
NUM_PERTURBED_MODELS = 100

In [67]:
data_source = "data/obermeyer/obermeyer_data_cleaned.csv"
prediction_output = "predictions/obermeyer/model_weight_perturbation_"+MODEL_CLASS+".csv"
target_variable = "cost_t"
features = ['dem_female', 'dem_age_band_18-24_tm1', 'dem_age_band_25-34_tm1', 'dem_age_band_35-44_tm1', 'dem_age_band_45-54_tm1',
            'dem_age_band_55-64_tm1', 'dem_age_band_65-74_tm1', 'dem_age_band_75+_tm1', 'hypertension_elixhauser_tm1', 'cost_dialysis_tm1',
            'cost_emergency_tm1', 'cost_home_health_tm1', 'cost_ip_medical_tm1', 'cost_ip_surgical_tm1', 'cost_laboratory_tm1',
            'cost_op_primary_care_tm1', 'cost_op_specialists_tm1', 'cost_op_surgery_tm1', 'cost_other_tm1', 'cost_pharmacy_tm1',
            'cost_physical_therapy_tm1', 'cost_radiology_tm1', 'gagne_sum_tm1']
other_variables = ['person_id', 'qualified_cost_25', 'qualified_cost_75', 'qualified_cost_50', 'qualified_gagne_1', 'qualified_gagne_2', 'qualified_gagne_3']

In [68]:
df = pd.read_csv(data_source)
X = df[features+other_variables]
y = df[target_variable]

In [69]:
def get_baseline_model_coefficients(X_train, y_train):
    if MODEL_CLASS == "lr":
        model = LinearRegression()
    model.fit(X_train, y_train)
    return model.coef_

In [73]:
def get_perturbed_weights(baseline_weights, epsilon, random_seed):
    np.random.seed(random_seed)
    random_vector = np.random.randn(*baseline_weights.shape)
    random_vector = random_vector / np.linalg.norm(random_vector)
    perturbation = np.random.uniform(0, epsilon) * random_vector
    w = baseline_weights + perturbation
    if np.linalg.norm(w - baseline_weights) > epsilon:
        print(f"Oh no! Got a distance of {np.linalg.norm(w - baseline_weights)}")
    return w

In [74]:
def get_predictions_columns(X_test, X_train, y_train, baseline_weights, epsilon, baseline_loss):
    predictions = []
    columns = []
    count = 0
    better = 0
    for i in tqdm(range(NUM_PERTURBED_MODELS)):
        perturbed_weights = get_perturbed_weights(baseline_weights, epsilon, i)
        y_pred = np.dot(X_test, perturbed_weights)
        train_pred = np.dot(X_train, perturbed_weights)
        rss_loss = np.sum((y_train - train_pred) ** 2)
        if rss_loss - baseline_loss > epsilon:
            # print(f"Uh oh! Got a RSS loss of {rss_loss}")
            count += 1
            # continue
        if rss_loss < baseline_loss:
            # print(f"perturbation {i} led to a better model")
            better += 1
        predictions.append(y_pred)
        columns.append(f'm_{i+1}')
    print(f"Number of models with higher loss than baseline + epsilon: {count}")
    print(f"Number of models with lower loss than baseline: {better}")
    return predictions, columns

In [75]:
output = []
for random_seed in range(NUM_RANDOM_SEEDS):
    print("random seed", random_seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random_seed)
    
    cost_threshold_25 = X_test['qualified_cost_25'].to_numpy()
    cost_threshold_50 = X_test['qualified_cost_50'].to_numpy()
    cost_threshold_75 = X_test['qualified_cost_75'].to_numpy()
    gagne_threshold_1 = X_test['qualified_gagne_1'].to_numpy()
    gagne_threshold_2 = X_test['qualified_gagne_2'].to_numpy()
    gagne_threshold_3 = X_test['qualified_gagne_3'].to_numpy()
    test_idx = X_test['person_id'].to_numpy()

    X_train = X_train.drop(columns=other_variables).to_numpy()
    y_train = y_train.to_numpy()
    X_test = X_test.drop(columns=other_variables).to_numpy()
    y_test = y_test.to_numpy()

    # orthonormalize - fix this!!
    intercept_idx = 0
    Xf_train = np.insert(X_train, intercept_idx, 1.0, axis=1)
    X_train_proj = sp.linalg.orth(Xf_train)
    Xf_test = np.insert(X_test, intercept_idx, 1.0, axis=1)
    X_test_proj = sp.linalg.orth(Xf_test)

    baseline_weights = get_baseline_model_coefficients(X_train_proj, y_train)
    baseline_prediction = np.dot(X_train_proj, baseline_weights)
    baseline_loss = np.sum((y_train - baseline_prediction) ** 2)
    print(f"baseline loss {baseline_loss}")
    predictions, columns = get_predictions_columns(X_test_proj, X_train_proj, y_train, baseline_weights, EPSILON, baseline_loss)

    predictions_df = pd.DataFrame(predictions).transpose()
    predictions_df.columns=columns
    predictions_df["y"] = y_test
    predictions_df["person_id"] = test_idx 
    predictions_df["seed"] = random_seed
    
    predictions_df['qualified_cost_25'] = cost_threshold_25
    predictions_df['qualified_cost_50'] = cost_threshold_50
    predictions_df['qualified_cost_75'] = cost_threshold_75
    predictions_df['qualified_gagne_1'] = gagne_threshold_1
    predictions_df['qualified_gagne_2'] = gagne_threshold_2
    predictions_df['qualified_gagne_3'] = gagne_threshold_3

    output.append(predictions_df)
    print()
output = pd.concat(output)

random seed 0
baseline loss 1.1776383451543227e+26


100%|██████████| 100/100 [00:00<00:00, 278.38it/s]


Number of models with higher loss than baseline + epsilon: 35
Number of models with lower loss than baseline: 43

random seed 1
baseline loss 2.3426159684741885e+27


100%|██████████| 100/100 [00:00<00:00, 372.17it/s]


Number of models with higher loss than baseline + epsilon: 18
Number of models with lower loss than baseline: 21

random seed 2
baseline loss 6.911289031031674e+24


...

Number of models with higher loss than baseline + epsilon: 50
Number of models with lower loss than baseline: 43

random seed 3
baseline loss 3.2117846351665025e+24


100%|██████████| 100/100 [00:00<00:00, 592.75it/s]


Number of models with higher loss than baseline + epsilon: 49
Number of models with lower loss than baseline: 48

random seed 4
baseline loss 9.112423597690497e+26


100%|██████████| 100/100 [00:00<00:00, 331.19it/s]


Number of models with higher loss than baseline + epsilon: 17
Number of models with lower loss than baseline: 29



In [76]:
output.head()

,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,...,m_100,y,person_id,seed,qualified_cost_25,qualified_cost_50,qualified_cost_75,qualified_gagne_1,qualified_gagne_2,qualified_gagne_3
0,1.038622e+11,1.038622e+11,1.038622e+11,1.038622e+11,1.038622e+11,1.038622e+11,1.038622e+11,1.038622e+11,1.038622e+11,1.038622e+11,...,1.038622e+11,0.009628,2545,0,0,1,1,1,0,0
1,9.403475e+10,9.403475e+10,9.403475e+10,9.403475e+10,9.403475e+10,9.403475e+10,9.403475e+10,9.403475e+10,9.403475e+10,9.403475e+10,...,9.403475e+10,0.004905,8198,0,0,0,1,1,1,0
2,8.716053e+10,8.716053e+10,8.716053e+10,8.716053e+10,8.716053e+10,8.716053e+10,8.716053e+10,8.716053e+10,8.716053e+10,8.716053e+10,...,8.716053e+10,0.009446,46461,0,0,1,1,0,0,0
3,7.495312e+10,7.495312e+10,7.495312e+10,7.495312e+10,7.495312e+10,7.495312e+10,7.495312e+10,7.495312e+10,7.495312e+10,7.495312e+10,...,7.495312e+10,0.002361,30620,0,0,0,1,0,0,0
4,9.909428e+10,9.909428e+10,9.909428e+10,9.909428e+10,9.909428e+10,9.909428e+10,9.909428e+10,9.909428e+10,9.909428e+10,9.909428e+10,...,9.909428e+10,0.003996,47418,0,0,0,1,1,1,0


In [77]:
output.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80495 entries, 0 to 16098
Columns: 109 entries, m_1 to qualified_gagne_3
dtypes: float64(101), int64(8)
memory usage: 67.6 MB


In [78]:
output.to_csv(prediction_output, index=False)